In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

%matplotlib inline
plt.style.use('bmh')

data_train = np.load('data_train_vars.npy')
data_train = data_train[:5500, :]
labels_train = np.load('labels_train_corr_vars.npy')
labels_train = labels_train[:5500]

VRAM_ALLOC = 8_000# in MB, up to 80_000 if you are using the A100 GPU
if tf.config.list_physical_devices('GPU'):
    for GPU in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_virtual_device_configuration(GPU,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=VRAM_ALLOC)])

def Train_model_accuracy(data_train, labels_train):

    class_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only','Do Not Enter','Crosswalk','Handicap Parking','No Parking']
    # Training and Test sets
    
    X_training, X_test, t_training, t_test = train_test_split(data_train, 
                                                  labels_train, 
                                                  shuffle=True,
                                                  stratify=labels_train,
                                                  test_size=0.15)
    # Train/Learn and validation sets
    
    X_train, X_val, t_train, t_val = train_test_split(X_training, 
                                                  t_training, 
                                                  shuffle=True,
                                                  stratify=t_training,
                                                  test_size=0.2)
    
    # cast numpy arrays into tensorflow tensors
    
    X_train_rs = tf.constant(X_train.reshape((X_train.shape[0],300,300,3)),
                        dtype=tf.float16)

    X_val_rs = tf.constant(X_val.reshape((X_val.shape[0],300,300,3)),
                        dtype=tf.float16)

    X_test_rs = tf.constant(X_test.reshape((X_test.shape[0],300,300,3)),
                        dtype=tf.float16)

    X_train_rs.shape, X_val_rs.shape, X_test_rs.shape
    
    X_train_rs = X_train_rs / 255.0
    X_val_rs = X_val_rs / 255.0
    X_test_rs = X_test_rs / 255.0

# =============================================================================================================================================

#Generate Model

    data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2)
    ])

    CNN = keras.models.Sequential([
        data_augmentation,
        keras.layers.Conv2D(64, 3, strides=2, activation='relu', padding='same', input_shape=[300,300,3]),
        keras.layers.Dropout(0.2),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(128, 3, strides=2,activation='relu', padding='same'),
        keras.layers.Conv2D(128, 3, strides=2,activation='relu', padding='same'),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(256, 3, strides=2,activation='relu', padding='same'),
        keras.layers.Conv2D(256, 3, strides=2,activation='relu', padding='same'),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(80, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=0.001, l2=0.001)),
        keras.layers.Dropout(.1),
        keras.layers.Dense(80, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=0.001, l2=0.001)),
        keras.layers.Dropout(.1),
        keras.layers.Dense(10, activation='softmax')
    ])

    CNN.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(0.001),
                 metrics=['accuracy'])

# ==============================================================================================================================================
#Train Model

    hist = CNN.fit(X_train_rs, t_train, epochs=50, batch_size=100,  #Use different training portion next time
             validation_data=(X_val_rs, t_val),
        
                   callbacks=[keras.callbacks.EarlyStopping(patience=30),
                       keras.callbacks.ModelCheckpoint('CNN.h5', save_best_only=True)])
# =============================================================================================================================================
#Fit Model to Training Data

    y_train = np.argmax(CNN.predict(X_train_rs), axis=1)
    accuracy_score_fnc_output = accuracy_score(t_train, y_train)

    return accuracy_score_fnc_output, y_train

In [2]:
train_accuracy ,y_train = Train_model_accuracy(data_train, labels_train)
print('Model Accuracy:',train_accuracy)
print('Predictions:', y_train)

Epoch 1/50
38/38 [==============================] - 9s 132ms/step - loss: 3.8871 - accuracy: 0.2203 - val_loss: 2.9762 - val_accuracy: 0.4267
Epoch 2/50
38/38 [==============================] - 4s 116ms/step - loss: 2.6483 - accuracy: 0.4537 - val_loss: 2.3550 - val_accuracy: 0.5433
Epoch 3/50
38/38 [==============================] - 4s 116ms/step - loss: 2.0839 - accuracy: 0.5775 - val_loss: 1.8097 - val_accuracy: 0.6332
Epoch 4/50
38/38 [==============================] - 4s 116ms/step - loss: 1.6028 - accuracy: 0.6885 - val_loss: 1.4087 - val_accuracy: 0.7914
Epoch 5/50
38/38 [==============================] - 4s 116ms/step - loss: 1.6116 - accuracy: 0.6610 - val_loss: 1.6444 - val_accuracy: 0.6406
Epoch 6/50
38/38 [==============================] - 4s 116ms/step - loss: 1.3990 - accuracy: 0.7128 - val_loss: 1.3071 - val_accuracy: 0.7594
Epoch 7/50
38/38 [==============================] - 4s 116ms/step - loss: 1.1737 - accuracy: 0.7957 - val_loss: 1.1030 - val_accuracy: 0.8160
Epoch 